In [5]:
import pandas as pd

comps = pd.read_csv('Competitions.csv')
notebooks = pd.read_csv('final_notebooks.csv')

In [19]:
len(notebooks['CompetitionId'].unique().tolist())

251

In [6]:
def unpack_array(arr):
    if isinstance(arr, list):
        return [item for sublist in arr for item in (sublist if isinstance(sublist, list) else [sublist])]
    else:
        return arr

In [7]:
import re

def remove_comments_imports_prints(code):
    # Split the code into lines
    lines = code.split('\n')
    # Create an empty list to store the filtered lines
    filtered_lines = []

    # Flags for block comments
    inside_block_comment = False

    # Iterate over each line
    for line in lines:
        # Handle block comments
        if "'''" in line:
            inside_block_comment = not inside_block_comment
            continue
        if inside_block_comment:
            continue

        # Remove lines that start with an import statement
        if line.strip().startswith("import") or line.strip().startswith("from"):
            continue
        # Remove lines that start with a print statement
        if line.strip().startswith("print"):
            continue
        if line.strip().startswith("plt"):
            continue
        if line.strip().startswith("warnings"):
            continue
        if line.strip().startswith("logging"):
            continue
        if line.strip().startswith("display"):
            continue
        if line.strip().startswith("filterwarnings"):
            continue
        if line.strip().startswith("sns"):
            continue
        if line.strip().startswith("random"):
            continue
        # Remove lines that are comments
        if re.match(r'\s*#', line):
            continue
        if re.match(r'\s*!', line):
            continue
        if re.match(r'\s*%', line):
            continue
        if re.match(r'.*\b(sns)\b.*', line):
            continue
        # Remove lines that contain plotting statements
        if re.match(r'.*\.(plot|show|scatter|hist|imshow|figure|sns)\(', line):
            continue
        # Add the line to the filtered lines list
        filtered_lines.append(line)

    # Join the filtered lines back into a single string
    filtered_code = '\n'.join(filtered_lines)

    # Remove multiple consecutive newline characters
    filtered_code = re.sub(r'\n{2,}', '\n', filtered_code)

    return filtered_code

In [22]:
good_notebooks = notebooks.dropna(subset=['code'])

In [23]:
len(good_notebooks['CompetitionId'].unique().tolist())

251

In [ ]:
good_notebooks['code'] = good_notebooks['code'].apply(lambda x: unpack_array(eval(x)))
good_notebooks['code'][1]

In [ ]:
clean_code = []


for i, row in good_notebooks.iterrows():
    full_code = '\n'.join(row['code'])
    clean_code.append(remove_comments_imports_prints(full_code))


good_notebooks['CleanCode'] = clean_code
good_notebooks

In [11]:
cl = []
for index, row in good_notebooks.iterrows():
    cl.append((index, row['CleanCode'].count('\n')))


In [12]:
from heapq import nsmallest, nlargest

print(f'Min: {nsmallest(30, cl, key=lambda item: item[1])}\nMax: {nlargest(5, cl, key=lambda item: item[1])}')

Min: [(218, 0), (237, 0), (420, 0), (511, 0), (912, 0), (1399, 0), (1849, 0), (2137, 0), (2584, 0), (2616, 0), (2877, 0), (3115, 0), (3147, 0), (3231, 0), (3274, 0), (3294, 0), (3482, 0), (3551, 0), (4219, 0), (4278, 0), (4422, 0), (5027, 0), (6035, 0), (6210, 0), (6786, 0), (6956, 0), (7003, 0), (7982, 0), (7988, 0), (8346, 0)]
Max: [(653, 3150), (18021, 3112), (4291, 3040), (8662, 2607), (15042, 2452)]


In [13]:
final = good_notebooks[good_notebooks['CleanCode'].str.count('\n') < 300]
final = final[good_notebooks['CleanCode'].str.count('\n') > 15]

/var/folders/zh/dsbs1wbd72z81b85y9td6sx00000gn/T/ipykernel_30211/4059254357.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final = final[good_notebooks['CleanCode'].str.count('\n') > 15]


In [14]:
final.to_csv('notebooks_meta.csv')

In [15]:
final['PrivateLeaderboardRank'] = final['PrivateLeaderboardRank'].astype(int)
final_mini = final.loc[final.groupby('CompetitionId')['PrivateLeaderboardRank'].idxmin()]

In [17]:
final_mini.to_csv('mini_meta.csv')